In [ ]:
//Import para Jupyter-notebooks 
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.Column
import org.apache.spark.sql.functions.udf

import java.time.LocalDate
import java.time.format.DateTimeFormatter

val spark = {
  SparkSession.builder()
    .master("local[1]")
    .getOrCreate()
}

import spark.implicits._

def toGroup: String => String = { bornDate =>
    val formatter = DateTimeFormatter.ofPattern("M/d/yyyy")
    val date = LocalDate.parse(bornDate, formatter)
    date.getYear match {
      case x if x < 1984              => "A"
      case x if x >= 1984 && x < 1990 => "B"
      case _                          => "C"
    }
}



In [ ]:
val people: DataFrame = spark.read
  .format("csv")
  .option("sep", ";")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("all.csv")

val toGroupUDF = udf(toGroup)

val city1Girls = people
  .where('city === "CITY1" && $"gender" === "F")
  .withColumn("group", toGroupUDF($"bornOn"))

val total = city1Girls.count()
val aGroup = city1Girls.where('group === "A").count()
val bGroup = city1Girls.where('group === "B").count()
val cGroup = city1Girls.where('group === "C").count()

println(s"CITY1: $total")
println(s"Group A: $aGroup, Group B: $bGroup, Group C: $cGroup")

city1Girls
  .select($"lastName", $"name", 'group)
  .orderBy("lastName")
  .show(20)